### Conversational AI - Research

In [ ]:
!pip install pgvector psycopg2 einops
!pip install sentencepiece

In [42]:
import torch
import torch.nn.functional as F

import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from transitions import Machine
from dotenv import dotenv_values
from rich import print


secret = dotenv_values('../../.secret')

gpt3_5_turbo  = dspy.OpenAI(
    model='gpt-3.5-turbo-0125',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

gpt4o  = dspy.OpenAI(
    model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)


llama3 = dspy.GROQ(
    model='llama3-70b-8192',
    api_key=secret['GROQ_API_KEY'],
    max_tokens=4096
)

dspy.settings.configure(lm=llama3)


def get_device():
    device = "cuda" \
        if torch.cuda.is_available() \
        else "mps" if torch.backends.mps.is_available() \
        else "cpu"
    if device == "mps":
        import os
        os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
        os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    return torch.device(device)


In [43]:
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel


tokenizer_embed = AutoTokenizer.from_pretrained('bert-base-uncased')
model_embed = AutoModel.from_pretrained('nomic-ai/nomic-embed-text-v1.5', trust_remote_code=True, safe_serialization=True)
model_embed.eval()


def embedd(text: str):
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


    encoded_input = tokenizer_embed(text, padding=True, truncation=True, return_tensors='pt')

    # + matryoshka_dim = 512

    with torch.no_grad():
        model_output = model_embed(**encoded_input)

    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # + embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
    # + embeddings = embeddings[:, :matryoshka_dim]
    embeddings = F.normalize(embeddings, p=2, dim=1)

    return np.array(embeddings)[0]



In [44]:
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

def persona_hub(persona_desc: str, subset: str) -> None:
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        
        register_vector(connection)
        cursor = connection.cursor()
        data = [
            (subset, persona_desc, embedd(text=persona_desc))
        ]

        execute_values(cursor, "INSERT INTO persona_hub (subset, persona_desc, persona_embedding) VALUES %s", data)
        connection.commit()


    except (Exception, psycopg2.Error) as error:
        print("Error while INSERTING data from PostgreSQL", error)

    finally:
        if connection:
            cursor.close()
            connection.close()



def retrieve_persona(text: str, K: int, negative_similarity: bool = False) -> list:
    personas = []
    try:
        connection = psycopg2.connect(user="drfadul",
                                    password="dROG@dijoFadul",
                                    host="localhost",
                                    port="5432",
                                    database="synaia")
        register_vector(connection)
        cursor = connection.cursor()
        text_embedd = embedd(text=text)
        data = (text_embedd, text_embedd, K)
    
        persona_hub_query = "SELECT subset, persona_desc, 1 - (persona_embedding <=> %s::vector) AS similarity FROM persona_hub ORDER BY persona_embedding <=> %s::vector LIMIT %s"
        if negative_similarity:
            persona_hub_query = "SELECT subset, persona_desc, 1 - (persona_embedding <=> %s::vector) AS similarity FROM persona_hub ORDER BY persona_embedding <=> %s::vector DESC LIMIT %s"

        cursor.execute(persona_hub_query, data)
        records = cursor.fetchall()

        for row in records:
            personas.append(
                (row[1], row[2])
            )
        
        return personas

    except (Exception, psycopg2.Error) as error:
        print("\x1b[1;31m Error while fetching data from PostgreSQL \x1b[1;31m", error)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()





In [45]:
from datasets import load_dataset
from tqdm import tqdm

sample_size = 200_000

# Load the dataset
persona_dataset = load_dataset("proj-persona/PersonaHub", data_files="persona.jsonl")['train']
persona_dataset = persona_dataset[:sample_size]
print(f"Total number of input personas: {len(persona_dataset['persona'])}")

Total number of input personas: 200000

In [25]:
if False:
    for persona in tqdm(persona_dataset['persona']):
        persona = persona.strip()
        persona_hub(persona, "persona")
    

100%|██████████| 200000/200000 [3:20:20<00:00, 16.64it/s]  


In [46]:
import torch
import torch.nn.functional as F
from transformers import AlbertTokenizer, AlbertForSequenceClassification
import random

# Define the model name (the name should correspond to the fine-tuned model you want to use)
model_name = "JanSt/albert-base-v2_mbti-classification"
device = get_device()

# Load the tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForSequenceClassification.from_pretrained(model_name)
model.eval()
model.to(device)


def get_personality_indicator(persona: str):
    # Tokenize the text
    inputs = tokenizer(persona, return_tensors='pt').to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted class
    logits = outputs.logits
    probabilities = F.softmax(logits, dim=1)
    probabilities_list = probabilities.squeeze().tolist()
    probabilities_list = {model.config.id2label[i]: p for i, p in enumerate(probabilities_list)}
    p_type = max(probabilities_list, key=lambda x: probabilities_list[x])

    return p_type, probabilities_list



In [47]:
from datasets import load_dataset
from tqdm import tqdm


# Load the dataset
mb_dataset = load_dataset("kl08/myers-briggs-type-indicator")['train']
print(f"Total number of myers-briggs records: {len(mb_dataset)}")

Repo card metadata block was not found. Setting CardData to empty.


Total number of myers-briggs records: 8675

In [48]:
import random
from rich import print

rdm = random.randint(0, len(mb_dataset))
min_len_post = 150 # chars
min_posts = 10
N = len(mb_dataset)
post_list = []

for p_type, posts in zip(mb_dataset['type'][:N], mb_dataset['posts'][:N]):
    posts = posts.split("|||")
    posts = {post for post in posts if "http" not in post and "www" not in post and len(post) > min_len_post}
    if len(posts) > min_posts:
        post_list.append(
            {
                "type": p_type,
                "posts": posts
            }
        )

len(post_list)

In [49]:
K = 500
key_words = [
    ['Young', 'Modern', 'Joyful', 'Outspoken',   'Inexperienced', "Open-minded", 'Smart', "Change-resistant", 'selfish', 'Honest', 'Relaxed']
]

__persona_applicant = []
for kw in [" ".join(k).strip() for k in key_words]:
    retrieved = retrieve_persona(text=kw, K=K)
    for r in retrieved:
        __persona_applicant.append(r[0])



key_words = [
    ["friendly", "knowledable", "professional", "women"],
    # ["racing", "adventurer", "rich", "job"],
]

__persona_recruitment = []
for kw in [" ".join(k).strip() for k in key_words]:
    retrieved = retrieve_persona(text=kw, K=K)
    for r in retrieved:
        __persona_recruitment.append(r[0])



In [60]:
import random

# rdm = random.randint(0, len(persona_dataset['persona']))
# persona = persona_dataset['persona'][rdm]

persona_applicant = random.choice(__persona_applicant)

p_type_applicant, probabilities_list = get_personality_indicator(persona_applicant)

persona_applicant_posts = random.choice([ p['posts'] for p in  post_list if p['type'] == p_type_applicant])



persona_recruitment = random.choice(__persona_recruitment)

p_type_recruitment, probabilities_list = get_personality_indicator(persona_recruitment)

persona_recruitment_posts = random.choice([ p['posts'] for p in  post_list if p['type'] == p_type_recruitment])


recruitment_posts = "\n".join(random.choices(list(persona_recruitment_posts), k=10))
applicant_posts = "\n".join(random.choices(list(persona_applicant_posts), k=10))


print(f"{p_type_applicant} -> {persona_applicant}")
# print(applicant_posts)

print(f"{p_type_recruitment} -> {persona_recruitment}")
# print(recruitment_posts)



ISTJ -> A talented young coder lured into the world of cybercrime by the promise of making a difference

INFJ -> a professional woman in STEM from Nigeria

In [33]:

prompt = f"""
Generate a conversational chitchat dialogue between an applicant and a recruiter (Maria).
All conversation is conducted through WhatsApp.
The position to be filled is as an agent in a call center.

Take into account the Personas and Myers–Briggs personality types indicator to enrich and make the dialogue more realistic:
- Applicant Persona: {persona_applicant}
- Applicant Indicator: {p_type_applicant}
- Recruiter Persona: {persona_recruitment}
- Recruiter Indicator: {p_type_recruitment}

Follow the guidances:
- The applicant initiates the conversation with a short greeting, it is possible that the applicant did not know the name of the recruiter.
- The recruiter's goal is to be proactive and guide the applicant through a series of tasks.
- The text includes metadata enclosed in pipe, You should NOT explicitly put this metadata in the generation.
- Craft a realistic and engaging conversational dialogue that simulates a real-world recruitment chat.
- Ensure the recruiter provides clear guidance and feedback between each task.
- Aim to reflect the authentic dynamics of a recruitment process, highlighting both the recruiter's professional advice and the applicant's genuine responses.
- No need to reiterate the information in the chat, as it's already captured in the submitted form. Let's move forward with the next steps!
- There is no need to transcribe voice notes in chat.
- Recruiter should be proactive.
- Recruiter should push the applicant to complete each step.
- In a new turn the applicant must explicitly indicate whether they completed a step or after sending a voice note.

Tasks to be completed during the chat:
1. Complete a basic form with required information, As a proactive recruiter, share the form.  |This step consists of the name, English level and accepting the terms and conditions. This form is shared within the chat|
2. Answer a two-question assessment within a form to evaluate your skills, it is not necessary to write the questions in the chat as they are included in the form,  As a proactive recruiter, share the form. |The applicant must answer two questions in two long paragraphs|
3. Record a voice note reading a provided text aloud and send it to the recruiter, As a proactive recruiter, share the text. |Write `PLACEHOLDER_1` intead of text, The text to be read is random and challenges the applicant's level of English.|
4. Respond to an open-ended question with a voice note,  allowing the recruiter to assess their thoughts and opinion, as a proactive recruiter, share the open-ended question. |Write `PLACEHOLDER_2` intead of text.|
"""

# llama3_output = llama3(prompt=prompt, temperature=0.5)
# llama3_output = " ".join([out for out in llama3_output])
# print(llama3_output)


gpt4o_output = gpt4o(prompt=prompt, temperature=0.5)
gpt4o_output = " ".join([out for out in gpt4o_output])
print(gpt4o_output)



|Applicant:| Hi there! I'm interested in the call center agent position. Could you help me with the application 
process?

|Recruiter (Maria):| Hello! Of course, I'd be happy to help you. My name is Maria. Let's start with the basic form.
Please fill out your name, English level, and accept the terms and conditions. Here is the form: [Form Link]

|Applicant:| I've completed the form. What's next?

|Recruiter (Maria):| Great! Now, let's move on to a two-question assessment to evaluate your skills. Please answer 
each question with a long paragraph. Here is the form: [Assessment Form Link]

|Applicant:| I've answered the questions. What should I do next?

|Recruiter (Maria):| Wonderful! Now, I need you to record a voice note reading a provided text aloud. This will 
help us assess your English pronunciation and fluency. Here is the text: `PLACEHOLDER_1`

|Applicant:| I've sent the voice note. What's the next step?

|Recruiter (Maria):| Excellent! Finally, I need you to respond to an open-ended question with a voice note. This 
will allow us to understand your thoughts and opinions. Here is the question: `PLACEHOLDER_2`

|Applicant:| I've sent the voice note with my response.

|Recruiter (Maria):| Thank you for completing all the steps. We will review your application and get back to you 
soon. Do you have any questions in the meantime?

|Applicant:| No, that's all for now. Thank you for your help!

|Recruiter (Maria):| You're welcome! Have a great day!

In [36]:
new_prompt = f"""
Given the following WhatsApp Chat Conversation enclosed in a triple backtick, to simulates a real-world recruitment chat perform the following:
- With 0.5 probability: between turns, incorporate the applicant's questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction, the Recruiter should assist the applicant in the new incorporated questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction.
- Restriction: NOT after last step have had completed, only one time between turns, the applicant expresses that they can't continue right now, are busy, or prefer to continue later. They might suggest any alternative way to stop the process and continue later. The Recruiter should assist the applicant by providing a form to schedule a time when the applicant can continue with the process. After the scheduling, simulate a time-elepsed and continue the conversation.
- Restriction: The Applicant cannot ask for a task that has already been completed, Sometimes the Applicant ask for resend of any specific form, task or question, reason for resend: the applicant may have missing it in the chat. 
- Regardless of the conversation flow, for each completed task there is a system notification with a message like this: ***System***: Task N completed, for a scheduled task: ***System***: Scheduled by the user.

About the process: Its take no more than 10 minutes to be completed. 
```{gpt4o_output}```
"""

new_prompt = f"""
Given the following WhatsApp Chat Conversation enclosed in a triple backtick, to simulates a real-world recruitment chat perform the following:
- With 0.5 probability: between turns, incorporate the applicant's questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction, the Recruiter should assist the applicant in the new incorporated questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction.
- NOT after last step have had completed, At least one time during the entire conversation, between turns, the applicant expresses that they don't want to continue or they are not interested anymore. The Recruiter should assist the applicant by providing a feedback and sharing a form to schedule a time if the applicant change their mind to continue with the process.
- Restriction: The Applicant cannot ask for a task that has already been completed, Sometimes the Applicant ask for resend of any specific form, task or question, reason for resend: the applicant may have missing it in the chat. 
- Regardless of the conversation flow, for each completed task there is a system notification with a message like this: ***System***: Task N completed, for a scheduled task: ***System***: Scheduled by the user.

About the process: Its take no more than 10 minutes to be completed. 
```{gpt4o_output}```
"""

new_prompt = f"""
Given the following WhatsApp Chat Conversation enclosed in a triple backtick, to simulates a real-world recruitment chat perform the following:
- With 0.5 probability: between turns, incorporate the applicant's questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction, the Recruiter should assist the applicant in the new incorporated questions, concerns, opinions, doubts, statements, and any other dialogue that may naturally arise during the interaction.
- Restriction: NOT after last step have had completed, in a certain turn, the applicant expresses that they don't want to continue or they are not interested anymore. The Recruiter should assist the applicant by providing a feedback and sharing a form to schedule a time if the applicant change their mind to continue with the process. The applicant does not want to continue and the process ends.
- Regardless of the conversation flow, for each completed task there is a system notification with a message like this: ***System***: Task N completed, for a scheduled task: ***System***: Scheduled by the user.

About the process: Its take no more than 10 minutes to be completed. 
```{gpt4o_output}```
"""

outputs = gpt4o(prompt=new_prompt, temperature=0.5)
outputs = " ".join([out for out in outputs])
print(outputs)

# print(new_prompt)




```|Applicant:| Hi there! I'm interested in the call center agent position. Could you help me with the application 
process?

|Recruiter (Maria):| Hello! Of course, I'd be happy to help you. My name is Maria. Let's start with the basic form.
Please fill out your name, English level, and accept the terms and conditions. Here is the form: [Form Link]

|Applicant:| I've completed the form. What's next?

***System***: Task 1 completed.

|Recruiter (Maria):| Great! Now, let's move on to a two-question assessment to evaluate your skills. Please answer 
each question with a long paragraph. Here is the form: [Assessment Form Link]

|Applicant:| I've answered the questions. What should I do next?

***System***: Task 2 completed.

|Recruiter (Maria):| Wonderful! Now, I need you to record a voice note reading a provided text aloud. This will 
help us assess your English pronunciation and fluency. Here is the text: `PLACEHOLDER_1`

|Applicant:| I've sent the voice note. By the way, how long does it usually take to get feedback?

|Recruiter (Maria):| Excellent! It usually takes about 1-2 business days for us to review your application and get 
back to you. Now, I need you to respond to an open-ended question with a voice note. This will allow us to 
understand your thoughts and opinions. Here is the question: `PLACEHOLDER_2`

|Applicant:| I've sent the voice note with my response.

***System***: Task 3 completed.

|Recruiter (Maria):| Thank you for completing all the steps. We will review your application and get back to you 
soon. Do you have any other questions in the meantime?

|Applicant:| Actually, I'm not sure if I want to continue with this process. I think I might not be interested 
anymore.

|Recruiter (Maria):| I understand. Thank you for your time and consideration. If you change your mind, please feel 
free to reach out. Here is a form to schedule a time if you decide to continue with the process: [Scheduling Form 
Link]

***System***: Task 4 completed.
```

In [41]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from transitions import Machine
from dotenv import dotenv_values
from rich import print


secret = dotenv_values('../../.secret')

gpt3_5_turbo  = dspy.OpenAI(
    model='gpt-3.5-turbo-0125',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

gpt4o_recruitment  = dspy.OpenAI(
    model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096,
    system_prompt=f"""You are a Recruiter (Maria), you conduct a recruiment process through WhatsApp, your Persona description: {persona_recruitment}.
    Your goal is to be proactive and guide the applicant through a series of tasks.
    Task includes metadata enclosed in pipe, You should NOT explicitly put this metadata in your responses.
    Recruiter should push the applicant to complete each step.
    """
)

gpt4o_applicant  = dspy.OpenAI(
    model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)


llama3 = dspy.GROQ(
    model='llama3-70b-8192',
    api_key=secret['GROQ_API_KEY'],
    max_tokens=4096
)

T = """Tasks to be completed during the chat:
1. Complete a basic form with required information, As a proactive recruiter, share the form.  |This step consists of the name, English level and accepting the terms and conditions. This form is shared within the chat|
2. Answer a two-question assessment within a form to evaluate your skills, it is not necessary to write the questions in the chat as they are included in the form,  As a proactive recruiter, share the form. |The applicant must answer two questions in two long paragraphs|
3. Record a voice note reading a provided text aloud and send it to the recruiter, As a proactive recruiter, share the text. |Write `PLACEHOLDER_1` intead of text, The text to be read is random and challenges the applicant's level of English.|
4. Respond to an open-ended question with a voice note,  allowing the recruiter to assess their thoughts and opinion, as a proactive recruiter, share the open-ended question. |Write `PLACEHOLDER_2` intead of text.|
"""


predict = dspy.Predict("applicant -> response")
with dspy.context(lm=gpt4o_recruitment):
    print(predict(applicant="Hi there"))


Prediction(
    response="Applicant: Hi there\nResponse: Hello! Thank you for reaching out. I'm Maria, and I'll be guiding you 
through the recruitment process. Could you please start by sharing your full name and the position you are applying
for?"
)